In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
import os
os.chdir('/fast/work/groups/ag_kircher/ReMM/MA_Lusi/Snakemake')
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [ ]:
!conda install -c conda-forge jupyter_contrib_nbextensions
!jupyter contrib nbextension install --user
!jupyter nbextension enable codefolding/main

!conda install -c conda-forge lo
!jupyter nbextensions_configurator enable --user

In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
import os

os.chdir('/fast/work/groups/ag_kircher/ReMM/MA_Lusi/Snakemake')
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [4]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression

In [6]:
a = 10

In [9]:
a

NameError: name 'a' is not defined

In [ ]:
os.cwd()

# Hesdfsdng

kljfs

### Data prearation

In [ ]:
df19 = pd.read_csv('output/features/annotated/hg19/SNVs.hg19.combined.txt.gz', sep = '\t',header = None)
df38 = pd.read_csv('output/features/annotated/hg38/SNVs.hg38.combined.txt.gz', sep = '\t',header = None)
lift= pd.read_csv('output/predictions/lifted/new/hg38.predictions.lifted.txt', sep = '\t',header = None)


df38.index= df38[1].astype(str)+'-'+(df38[2]).astype(str)
df19.index= df19[1].astype(str)+'-'+(df19[2]).astype(str)
#print(df19.memory_usage().sum()/a,df38.memory_usage().sum()/a,lift.memory_usage().sum()/a)

col = pd.read_csv('input/variants/hg38/SNVs.hg38.positive.annotated.tsv.gz', sep = '\t')
#df38 = df38.drop(columns = [0])
df38.columns  = ['Score']+col.columns.str.strip().to_list()
df38['ID']=list(range(0,len(df38)))

col = pd.read_csv('input/variants/hg19/SNVs.hg19.positive.annotated.tsv.gz', sep = '\t')
#df19 = df19.drop(columns = [0])

df19.columns  = ['Score']+col.columns.str.strip().to_list()

df19.columns = df19.columns.str.replace('46way','')
df19.columns = df19.columns.str.replace('fantom','Fantom')

df38 = df38.fillna(0)
df19 = df19.fillna(0)

p = lift[2].str.split(':', expand = True)[1].astype(int)
lift.index = lift[2].str.split(':', expand = True)[0]+'-'+p.astype(str)

f = df38.join(lift,lsuffix='_38',rsuffix='_lift',how='inner')
f.index= f[0].astype(str)+'-'+(f[1]).astype(str)

ff = f.join(df19,lsuffix='_38',rsuffix='_19',how='inner')


fd = ff.drop_duplicates('ID')
fd = fd.drop(columns = ['CHR_19', 'POSITION_19','CHR_38', 'POSITION_38',0,1,2])

#fd.to_csv('output/predictions/lifted/new/ff.dataframe.txt',sep = '\t',index = True,header = True)




In [11]:
a = 10

In [ ]:
# without normalization 
#f = fd
#col = pd.read_csv('input/variants/hg38/SNVs.hg38.positive.annotated.tsv.gz', sep = '\t')
#cols = col.columns[3:]

In [ ]:
f = pd.read_csv('/data/groups/ag_kircher/ReMM/MA_Lusi/SingleFeatureImpact/cleaned_scaled_combined_annotations.tsv.gz',sep='\t')




In [4]:
# normalize and save the data 
from sklearn import preprocessing

cols = fd.columns

f = preprocessing.normalize(fd,axis = 0)
f = pd.DataFrame(f)
f.columns = cols

f['Score_38'] =fd.reset_index()['Score_38']
f['Score_19'] =fd.reset_index()['Score_19']


col = pd.read_csv('input/variants/hg38/SNVs.hg38.positive.annotated.tsv.gz', sep = '\t')
cols = col.columns[3:]
cols
#f.to_csv('output/predictions/lifted/new/ff.dataframe.normiert.txt',sep = '\t',index = True,header = True),float_format='%.5f'

NameError: name 'fd' is not defined

In [ ]:
(fd['GCContent_38'] - fd['GCContent_38'].mean())/fd['GCContent_38'].std()

In [ ]:
f0['GCContent_38']

In [ ]:
f1['GCContent_38']

In [ ]:
f0.head()

### Read in normalized data

In [ ]:
df19 = f.iloc[:,f.columns.str.contains('_19',na = False)]
df19.columns = df19.columns.str.replace('_19','')
df38 = f.iloc[:,f.columns.str.contains('_38',na = False)]
df38.columns = df38.columns.str.replace('_38','')

In [ ]:
df19.corrwith(df38)

In [ ]:
f=pd.read_csv('output/predictions/lifted/new/ff.dataframe.normiert.txt',sep = '\t',index_col = 'Unnamed: 0')



In [ ]:

col = pd.read_csv('input/variants/hg38/SNVs.hg38.positive.annotated.tsv.gz', sep = '\t')
cols = col.columns[3:]

##### 

In [ ]:
f['GerpRSpv_19'].value_counts() # normalized 

In [ ]:
f['GerpRS_19'].value_counts() # normalized

In [ ]:
ff['GerpRSpv_19'].value_counts() # not normalized

In [ ]:
ff['GerpRS_19'].value_counts() # not normalized

### Devide the dataset into positive and negative dataframes

In [ ]:
f['Label_19']=f['Label_38']

In [ ]:

f = f.reset_index(drop =True)
p = f[f['Label_38']==1]
n = f[f['Label_38']==0]

In [10]:
a

NameError: name 'a' is not defined

In [ ]:
n.loc[:,'Fantom5Robust_19'].max()

In [ ]:
#f = f.apply(lambda x: x.round(6))
#f.head()

### Run linear and logistic regressoins, calculate correlations and means of feature values 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from scipy.spatial import distance
import random

# dataframes to append lists with importance and correaltions
# each column is a list from one run (in total 100 for each genome build)
logistic = pd.DataFrame()
distances = pd.DataFrame()
linear = pd.DataFrame()
correlations = pd.DataFrame()

means_pos = pd.DataFrame()
means_neg = pd.DataFrame()

for i in range (0,100):
    # sample 406 random data points from the negative set
    s = random.sample(range(0, len(f)),406)
    df = n[n.index.isin(s)]
    # append the positive dataset
    df = df.append(p)
    
    # list of dataframes with 802 data points to calculate the euclidean distance 
    frames = []
    
    for a in ['_19','_38']:
        
        # lists to append importance and correlation values vor each iteration
        linear_list = []
        logistic_list = []
        correlations_list = []
        distances_list = []
        # filter for hg19 or hg38
        df_a = df.iloc[:,df.columns.str.contains(a,na = False)]
        df_a.columns = df_a.columns.str.replace(a,'')

        # devide into positive and negatove for feature means
        df_a_pos = df_a.loc[df_a['Label']==1,cols]
        df_a_neg = df_a.loc[df_a['Label']==0,cols]
        
        
        # calculate feature means 
        means_pos[str(i)+a] = df_a_pos.mean(axis = 0)
        means_neg[str(i)+a] = df_a_neg.mean(axis = 0)

        frames.append(df_a)
        
        for column in cols:
            # choose the column for the univariate analysis 
            X = df_a.loc[:,column].to_numpy().reshape(-1, 1)
            y = df_a['Label']
            
            # calculate the linear model
            model = LinearRegression().fit(X, y)
            importance = model.coef_[0]
            linear_list.append(importance)
            
            # calculate the logistic model
            model = LogisticRegression().fit(X, y)
            importance = model.coef_[0]
            logistic_list.append(importance[0])
            
            # calculate the correlation between scores (y) and feature (X)
            # c is a df with two columns, df.corr() returns a correlation matrix
            c = pd.DataFrame([list(X.flatten()),y]).T.corr().iloc[1,0]
            correlations_list.append(c)
            
           
        # lists are columns of final dataframes
        linear[str(i)+a] = linear_list
        logistic[str(i)+a] = logistic_list
        correlations[str(i)+a] = correlations_list
        
    # calculate euclidean distance     
    for column in cols: 
        dist = distance.euclidean(frames[0].loc[:,column], frames[1].loc[:,column])
        distances_list.append(dist)

    distances[str(i)+a] = distances_list




            



In [ ]:
correlations.head()

In [7]:
def kop(tog,name):        
    # input: dataframe with results of the loop (see example above)
    # output: a dataframe with two columns (hg19 and hg38) with means from the runs
    
    
    a = tog.loc[:,tog.columns.str.contains('_19')].mean(axis = 1)
    b = tog.loc[:,tog.columns.str.contains('_38')].mean(axis = 1)

    alle = pd.DataFrame([a,b]).T
    alle['index'] = cols#.append(cols)
    alle = alle.set_index('index')

    alle.index.name=''
    alle.columns = ['ReMM '+name,'parSMURF '+name]
    return  alle

In [ ]:
kop(correlations, 'Correlation')

In [ ]:

# join all the means in one table
al = kop(linear,'Linear').join(kop(logistic,'Logistic')).join(kop(means_pos, 'Positive Mean'))
al = al.join(kop(means_neg, 'Negative Mean')).join(kop(correlations, 'Correlation'))




In [ ]:
# add euclidean distance
d = distances.mean(axis = 1)
d.name = 'Euclidean Distance'
d.index = cols
al= al.join(d)

al['Euclidean Distance'] = al['Euclidean Distance'].astype(float).round(5)

In [8]:
col = pd.read_csv('input/variants/hg38/SNVs.hg38.positive.annotated.tsv.gz', sep = '\t')


In [ ]:
# add parSMURF Tree means
col = pd.read_csv('input/variants/hg38/SNVs.hg38.positive.annotated.tsv.gz', sep = '\t')


imp = pd.DataFrame()
for i in range(99):
    df = pd.read_csv('models/hg38/'+str(i)+'.out.importance',sep = ' ',header = None)
    imp[i]=df[1]

imp.index = col.columns[3:]
imp = imp.mean(axis = 1)
imp = pd.DataFrame(imp)


imp.columns = ['ParSMURF Tree']
df_order = pd.DataFrame(imp).join(al)

df_order.head()


In [ ]:
# order of values: order descending, add a columns with list from 1 to 26
for c in df_order.columns:
    df_order = df_order.sort_values(by=[c], ascending = False)
    df_order[c+' Order'] = list(range(1,27))


In [ ]:
#df_order.iloc[:,-12:]

In [ ]:
# df_order[['ReMM Linear Order','ReMM Correlation Order','parSMURF Linear Order','parSMURF Correlation Order']].sort_values(by=['parSMURF Linear Order'], ascending = True)




In [ ]:
df =pd.read_csv('output/predictions/hg19/remm_with_ps_predictions.csv',sep ='\t',compression = 'gzip',header = None)
var = pd.read_csv('output/features/annotated/hg19/SNVs.hg19.combined.txt.gz',sep = '\t', compression = 'gzip',header = None, usecols = [0,1,2])
ps = pd.read_csv('output/predictions/hg19/SNVs.hg19.predictions.txt', sep = '\t',header = None)

var['prediction']=ps[0]
df['id']=df[0]+'-'+df[1].astype(str)
var['id']=var[1]+'-'+var[2].astype(str)
var = var.set_index('id')
df = df.set_index('id')

fc = var.join(df, how='inner',lsuffix = 'ps',rsuffix='remm')

fc[['prediction','2remm']].corr()

In [ ]:
import random 
df = fc.reset_index()
a = []
for i in range(10):
    s = random.sample(range(0, len(fc)),100000)
    df0 = df[df.index.isin(s)]
    print(df0.shape)
    a.append(df0[['prediction','2remm']].corr().iloc[0,1])

In [5]:
print('Hallo0')


Hallo0


In [ ]:

def getImportance(hg): 
    file  = 'input/variants/'+hg+'/SNVs.'+hg+'.positive.annotated.tsv.gz'
    #print(file)
    col = pd.read_csv(file, sep = '\t')


    imp = pd.DataFrame()
    for i in range(99):
        df = pd.read_csv('models/'+hg+'/'+str(i)+'.out.importance',sep = ' ',header = None)
        imp[i]=df[1]

    c=3
    if hg=='hg19':
        c = 2
        col.columns = col.columns.str.replace('46way','').str.replace('fantom','Fantom')
    
    imp.index = col.columns[c:]
    imp = imp.mean(axis = 1)
    imp = pd.DataFrame(imp)
    imp.columns = ['ParSMURF Tree']
    return imp.round(5)


df_order = getImportance('hg38').join(getImportance('hg19'),lsuffix = '_hg38',rsuffix = '_hg19')


In [ ]:
dfs = []
for a in ['_19','_38']:
    df_a = f.iloc[:,f.columns.str.contains(a,na = False)]
    df_a.columns = df_a.columns.str.replace(a,'')
    dfs.append(df_a)
a = dfs[0].corrwith(dfs[1])
a = pd.DataFrame(a)
a.columns = ['Correlation']
df_order = df_order.join(a,how ='inner')


In [ ]:
for c in df_order.columns:
    df_order = df_order.sort_values(by=[c], ascending = False)
    df_order[c+' Order'] = list(range(1,27))

In [ ]:
m = df_order[['ParSMURF Tree_hg38','ParSMURF Tree_hg19']].max().min()
m

In [ ]:
df_order.sort_values(by=['ParSMURF Tree_hg38'], ascending = False).to_latex('figures/importance.latex')

In [ ]:
x = 2


feature = 'ParSMURF Tree_hg38'
features = ['ParSMURF Tree_hg19','ParSMURF Tree_hg38']

for feature in features:
    a = df_order.sort_values(by=[feature], ascending = True)

    #a['group'] = a['group'].map(colors)

    #colors = {0:'teal',1:'lightblue',2:'steelblue',3:'lightsteelblue'}
    
    fig,ax = plt.subplots(figsize=(16,9))


    ax.barh(a.index, a.loc[:,feature],color = a.loc[:,'color'])
    ax.set_xlabel('Importance', fontsize = 10*x,labelpad=15)
    #ax.set_ylabel('Feture',fontsize = 10*x,labelpad=10)
    ax.tick_params(axis='both',  labelsize=8*x)
    #ax.set_xlim([0, 0.5])
    #ax.set_xbound(lower=0.0, upper=m)
    ax.autoscale(enable=True, axis='y')
    plt.legend(*zip(*[(plt.Rectangle((0,0), 1,1, color=c), f"{i}") for i,c in colors.items()]),fontsize = 9*x,loc = 4) 
    plt.tight_layout()

    plt.savefig('figures/'+feature+'_bar.png',di=30,bbox_inches='tight')



In [ ]:
df_order['group'] ='Epigenetic profile'
ind = df_order.index[df_order.index.str.contains('GC|CpG')].to_list()
df_order.loc[ind,'group']='GC-based'
ind = df_order.index[df_order.index.str.contains('PhastCons|PhyloP|Gerp')].to_list()
df_order.loc[ind,'group']='Conservation and constraint'
ind = df_order.index[df_order.index.str.contains('Var|VAR|ISCApath|fracRareCommon|dbVARCount|DGVCount')].to_list()
df_order.loc[ind,'group']='Genetic variation'
colors = {'Epigenetic profile':'teal','GC-based':'lightblue','Conservation and constraint':'steelblue','Genetic variation':'lightsteelblue'}
#a = df_order.sort_values(by=[feature], ascending = True)

df_order['color'] = df_order['group'].map(colors)

In [ ]:
colors = {'Epigenetic profile':'teal','GC-based':'lightblue','Conservation and constraint':'steelblue','Genetic variation':'lightsteelblue'}
#colors = {'teal':0,'lightblue':1,'steelblue':2,'lightsteelblue':3}

In [ ]:
a = df_order.sort_values(by=[feature], ascending = True)

a['color'] = a['group'].map(colors)

In [ ]:
a#.loc[:,'color']

In [ ]:
df_order

In [ ]:

feature = 'ParSMURF Tree_hg19'
a = df_order.sort_values(by=[feature], ascending = False)
a = a.loc[:,feature]
a